In [ ]:
from PyQt5.QtWidgets import QApplication, QLabel, QMainWindow, QVBoxLayout, QWidget, QPushButton, QMessageBox, QInputDialog
from PyQt5.QtGui import QPixmap
from PyQt5.QtCore import Qt
import sys
import os
import numpy as np

coordinate_list = []  # List to store the coordinates
a_value = ""
h_value = ""

# Define functions to convert between logarithmic and pixel values
def pixel_to_log(x, y, width, height):
    log_x = x/1200 * (np.log10(600) - np.log10(6)) + np.log10(6)  # Convert x pixel to log
    log_y = y/800 * np.log10(100)  # Convert y pixel to log
    return log_x, log_y

def log_to_pixel(log_x, log_y, width, height):
    x = 10**(log_x) # Convert log back to x pixel
    y = 10**(log_y) # Convert log back to y pixel
    return x, y

def on_click(event):
    x, y = event.x(), label.height() - event.y()
    print(f"Clicked at (X: {x}, Y: {y})")
    # Convert pixel coordinates to logarithmic values
    log_x, log_y = pixel_to_log(x, y, label.width(), label.height())
    print(f"Clicked at (Log_X: {log_x:.2f}, Log_Y: {log_y:.2f})")
    coordinate_list.append((log_x, log_y))

def save_coordinates():
    global a_value, h_value
    if coordinate_list:
        a_value, a_ok = QInputDialog.getText(window, "Enter 'a'", "Enter the value of 'a':")
        if a_ok:
            h_value, h_ok = QInputDialog.getText(window, "Enter 'h'", f"Enter the value of 'h' for a={a_value}:")
            if h_ok:
                # Increment the file number
                file_number = len([name for name in os.listdir('.') if os.path.isfile(name) and 'data' in name]) + 1
                filename = f"data({file_number}).csv"
                
                # Delete previous data file if exists
                if os.path.exists(filename):
                    os.remove(filename)

                # Write coordinates to the CSV file in 10^x and 10^y format
                with open(filename, 'w') as file:
                    file.write(f"h={h_value},a={a_value}\n")
                    for coordinates in coordinate_list:
                        log_x, log_y = coordinates
                        x, y = (log_x), (log_y)  # Convert to log for 10^x and 10^y
                        file.write(f"{10**x},{10**y}\n")
                print(f"Coordinates saved to {filename}")
                coordinate_list.clear()
    else:
        QMessageBox.warning(window, "No Coordinates", "No coordinates to save.")

def finish_application():
    sys.exit()

def main():
    global window
    app = QApplication(sys.argv)
    window = QMainWindow()
    window.setWindowTitle("Click on the Image")

    # Load the image
    image_path = 'your_image_Cropped.png'  # Replace with the path to your image file
    pixmap = QPixmap(image_path)
    pixmap = pixmap.scaled(1200, 800)  # Resize the image to fit max X and Y axis

    # Create a label to display the image
    global label
    label = QLabel()
    label.setPixmap(pixmap)
    label.setAlignment(Qt.AlignTop)
    label.mousePressEvent = on_click

    # Create Save and Finish buttons
    save_button = QPushButton("Save")
    finish_button = QPushButton("Finish")

    # Connecting button actions to functions
    save_button.clicked.connect(save_coordinates)
    finish_button.clicked.connect(finish_application)

    # Layout for buttons and image label
    layout = QVBoxLayout()
    layout.addWidget(label)
    layout.addWidget(save_button)
    layout.addWidget(finish_button)

    # Create a central widget and set layout
    central_widget = QWidget()
    central_widget.setLayout(layout)

    # Set the central widget of the window
    window.setCentralWidget(central_widget)

    # Show the Widget
    window.show()
    sys.exit(app.exec_())

if __name__ == '__main__':
    main()

Clicked at (X: 1183, Y: 35)
Clicked at (Log_X: 2.75, Log_Y: 0.09)


In [5]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# Load and preprocess data for training
combined_X = []
combined_y = []

for i in range(1, 10):  # Load data(1).csv to data(9).csv for training
    file_name = f"data({i}).csv"
    df = pd.read_csv(file_name)
    # Assuming 'a' and 'h' are numeric values in the first row
    a_value = float(df.columns[0].split('=')[1])
    h_value = float(df.columns[1].split('=')[1])
    # Assuming 'n' is in the second column and 'ThermalConduction' in the third column
    X = df.iloc[:, 0].values.reshape(-1, 1)  # 'n' values
    y = df.iloc[:, 1].values  # Thermal conduction values
    combined_X.extend(X)
    combined_y.extend(y)

# Train your model using combined data
scaler = StandardScaler()
X_combined_scaled = scaler.fit_transform(combined_X)
regressor = MLPRegressor(hidden_layer_sizes=(100,15,35), activation='relu', solver='adam', max_iter=50000)

regressor.fit(X_combined_scaled, combined_y)



# Test data(10).csv to data(12).csv
for i in range(11, 14):
    file_name = f"data({i}).csv"
    df = pd.read_csv(file_name)
    # Assuming 'a' and 'h' are numeric values in the first row
    a_value = float(df.columns[0].split('=')[1])
    h_value = float(df.columns[1].split('=')[1])
    # Assuming 'n' is in the second column
    X_test = df.iloc[:, 0].values.reshape(-1, 1)  # 'n' values
    y_true = df.iloc[:, 1].values  # Actual Thermal conduction values if available
    # Using the model trained on combined data to predict thermal conduction
    X_test_scaled = scaler.transform(X_test)
    thermal_conduction_prediction = regressor.predict(X_test_scaled)
    print(f"Estimated Thermal Conduction for {file_name}: {thermal_conduction_prediction}")
    # Calculate Mean Squared Error if actual values are available
    if y_true is not None:
        mse = mean_squared_error(y_true, thermal_conduction_prediction)
        print(f"Mean Squared Error for {file_name}: {mse}")

Estimated Thermal Conduction for data(11).csv: [10.64078993 12.21231481 13.81165776 15.10338718 16.20966433 17.71734421
 20.66915096 22.47012759 24.28677366]
Mean Squared Error for data(11).csv: 21.43213910393773
Estimated Thermal Conduction for data(12).csv: [10.94294927 12.36770096 13.34902537 14.43721628 14.96023232 16.37214665
 17.54639838 18.4812795 ]
Mean Squared Error for data(12).csv: 10.940536931690351
Estimated Thermal Conduction for data(13).csv: [ 9.6803268   9.73302923  9.76109674  9.8798553   9.88600202  9.8984379
  9.93119192 10.00504135 10.00504135 10.22189308 10.33709351 10.9845674 ]
Mean Squared Error for data(13).csv: 31.411343553333438


In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Initialize empty arrays for combined data
combined_X = np.array([])
combined_y = np.array([])
combined_h = np.array([])
combined_a = np.array([])

# Loop through data files to extract and combine the data
for i in range(1, 10):
    file_name = f"data({i}).csv"
    df = pd.read_csv(file_name)

    # Extract 'a' and 'h' values from the file
    a_value = float(df.columns[0].split('=')[1])
    h_value = float(df.columns[1].split('=')[1])

    # Extract 'X' and 'y' values
    X = df.iloc[1:, 0].values.astype(float)
    y = df.iloc[1:, 1].values.astype(float)

    # Update 'a' and 'h' arrays to match the number of 'X' and 'y' rows
    h_values = np.full((len(X), 1), h_value)
    a_values = np.full((len(X), 1), a_value)

    # Combine 'X', 'y', 'h', and 'a' arrays
    if combined_X.size == 0:
        combined_X = X
        combined_y = y
        combined_h = h_values
        combined_a = a_values
    else:
        combined_X = np.concatenate((combined_X, X))
        combined_y = np.concatenate((combined_y, y))
        combined_h = np.concatenate((combined_h, h_values))
        combined_a = np.concatenate((combined_a, a_values))

# Reshape 'X' and 'y' arrays
combined_X = combined_X.reshape(-1, 1)
combined_y = combined_y.reshape(-1, 1)

# Concatenate 'X', 'y', 'h', and 'a' arrays to form the unified dataset
unified_data = np.concatenate((combined_X, combined_y, combined_h, combined_a), axis=1)

# Display the shape of the unified dataset
print(f"Shape of unified dataset: {unified_data.shape}")


# Split the unified data into features (X) and labels (y)
X = unified_data[:, [0,2,3]]  # Feature: First column as 'X'
y = unified_data[:, 1]  # Label: Second column as 'y'


# Define and compile the neural network model using Keras
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(3,)),
    tf.keras.layers.Dense(64, activation='selu'),
#    tf.keras.layers.Conv1D(32, 3, activation='relu'),  # Conv1D layer inserted here
    tf.keras.layers.Flatten(),    
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)  # Output layer for 'y'
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X, y, epochs=500, batch_size=3, verbose=1)

Shape of unified dataset: (39, 4)
Epoch 1/500
13/13 [==============================] - 1s 2ms/step - loss: 384.0249 - mae: 15.5609
Epoch 2/500
13/13 [==============================] - 0s 2ms/step - loss: 138.4131 - mae: 9.3794
Epoch 3/500
13/13 [==============================] - 0s 2ms/step - loss: 122.6335 - mae: 8.4524
Epoch 4/500
13/13 [==============================] - 0s 2ms/step - loss: 119.0854 - mae: 8.3948
Epoch 5/500
13/13 [==============================] - 0s 2ms/step - loss: 94.6286 - mae: 7.5804
Epoch 6/500
13/13 [==============================] - 0s 2ms/step - loss: 65.0227 - mae: 6.8504
Epoch 7/500
13/13 [==============================] - 0s 1ms/step - loss: 40.4992 - mae: 5.2691
Epoch 8/500
13/13 [==============================] - 0s 2ms/step - loss: 27.7813 - mae: 4.2181
Epoch 9/500
13/13 [==============================] - 0s 1ms/step - loss: 26.7263 - mae: 4.0583
Epoch 10/500
13/13 [==============================] - 0s 2ms/step - loss: 17.5622 - mae: 3.0660
Epoch 11/5